# Необходимые библиотеки

In [84]:
# !pip install cohere

import pandas as pd
import numpy as np
import cohere
import time

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Инициализация клиента Cohere с использованием API ключа
co = cohere.Client('DtT9RQFD24TXDE6VpgzenjRHx7PnL7A6loRjUd1V')

# Обработка данных

## Просто загрузка всех датасетов

In [85]:
splits = {'train': 'data/train-00000-of-00001-d1de06d6b6482a7a.parquet', 'dev': 'data/dev-00000-of-00001-6014228512208659.parquet', 'testA': 'data/testA-00000-of-00001-78a4a011ae1dc086.parquet', 'testB': 'data/testB-00000-of-00001-003543ac41b77cc0.parquet'}
df_train = pd.read_parquet("hf://datasets/Cohere/miracl-ru-queries-22-12/" + splits["train"])
df_dev = pd.read_parquet("hf://datasets/Cohere/miracl-ru-queries-22-12/" + splits["dev"])
df_testA = pd.read_parquet("hf://datasets/Cohere/miracl-ru-queries-22-12/" + splits["testA"])
df_testB = pd.read_parquet("hf://datasets/Cohere/miracl-ru-queries-22-12/" + splits["testB"])

In [86]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4683 entries, 0 to 4682
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query_id           4683 non-null   object
 1   query              4683 non-null   object
 2   positive_passages  4683 non-null   object
 3   negative_passages  4683 non-null   object
 4   emb                4683 non-null   object
dtypes: object(5)
memory usage: 183.1+ KB


In [87]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1252 entries, 0 to 1251
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query_id           1252 non-null   object
 1   query              1252 non-null   object
 2   positive_passages  1252 non-null   object
 3   negative_passages  1252 non-null   object
 4   emb                1252 non-null   object
dtypes: object(5)
memory usage: 49.0+ KB


In [88]:
df_testA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 911 entries, 0 to 910
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query_id           911 non-null    object
 1   query              911 non-null    object
 2   positive_passages  911 non-null    object
 3   negative_passages  911 non-null    object
 4   emb                911 non-null    object
dtypes: object(5)
memory usage: 35.7+ KB


In [89]:
df_testB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   query_id           718 non-null    object
 1   query              718 non-null    object
 2   positive_passages  718 non-null    object
 3   negative_passages  718 non-null    object
 4   emb                718 non-null    object
dtypes: object(5)
memory usage: 28.2+ KB


Количество сэмплов сходится с заявленным на странице датасета
(на последний номер строки данных на сайте смотреть, видимо, не стоит, объектов 4.68k, 1.25k, 911, 718)

## Преобразуем данные в нормальный вид

In [90]:
# Функция для создания строк с query, passage и label
def create_rows_from_passages(row):
    new_rows = []
    query = row['query']
    query_emb = row['emb']

    # Обработка positive_passages
    for pos_passage in row['positive_passages']:
        new_rows.append({
            'query_text': query,
            'passage_text': pos_passage['text'],
            'query_emb': query_emb,
            'label': 1
        })

    # Обработка negative_passages
    for neg_passage in row['negative_passages']:
        new_rows.append({
            'query_text': query,
            'passage_text': neg_passage['text'],
            'query_emb': query_emb,
            'label': 0
        })

    return new_rows

In [91]:
# Применение функции ко всему DataFrame
new_rows = []
for index, row in df_train.iterrows():
    new_rows.extend(create_rows_from_passages(row))

# Создание нового DataFrame
df_train = pd.DataFrame(new_rows)

# Применение функции ко всему DataFrame
new_rows = []
for index, row in df_dev.iterrows():
    new_rows.extend(create_rows_from_passages(row))

# Создание нового DataFrame
df_test = pd.DataFrame(new_rows)

In [73]:
# Проверим, как выглядит DataFrame
df_train.head()

,query_text,passage_text,query_emb,label
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",1
1,Когда был спущен на воду первый миноносец «Спо...,Стерегу́щий — русский миноносец типа «Сокол». ...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0
2,Когда был спущен на воду первый миноносец «Спо...,Эскадренный миноносец заложен в 1900 году на Н...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0
3,Когда был спущен на воду первый миноносец «Спо...,10 октября 1937 года эсминец был спущен на вод...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0
4,Когда был спущен на воду первый миноносец «Спо...,В 1901 году миноносец «Бодрый» был зачислен в ...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0


## Закодируем пассажи тем же embedder - Cohere-multilingual-22-12

In [80]:
# Функция для получения эмбеддингов для одного батча
def get_embeddings_batch(batch_texts):
    response = co.embed(texts=batch_texts, model="multilingual-22-12")
    return response.embeddings

# Разбиваем тексты на батчи по 100
def process_in_batches(texts, batch_size=500, wait_time=65):
    num_batches = len(texts) // batch_size + (1 if len(texts) % batch_size != 0 else 0)
    all_embeddings = []

    for i in range(num_batches):
        batch_texts = texts[i * batch_size:(i + 1) * batch_size]
        print(f"Processing batch {i+1}/{num_batches}")

        # Получаем эмбеддинги для текущего батча
        embeddings = get_embeddings_batch(batch_texts)
        all_embeddings.extend(embeddings)

        # Ждем 1 минуту и 5 секунд после обработки каждого батча, кроме последнего
        if i < num_batches - 1:
            print(f"Waiting {wait_time} seconds before next batch...")
            time.sleep(wait_time)

    return all_embeddings

In [ ]:
train_passages = df_train['passage_text'].tolist()
embeddings_train = process_in_batches(train_passages)

# Сохраняем эмбеддинги в файл
output_file = "passage_embeddings_train.npy"
np.save(output_file, embeddings_train)

In [ ]:
test_passages = df_test['passage_text'].tolist()
embeddings_test = process_in_batches(test_passages)

# Сохраняем эмбеддинги в файл
output_file = "passage_embeddings_test.npy"
np.save(output_file, embeddings_test)

## Финальный вид датасетов

In [92]:
embeddings_train = list(np.load('passage_embeddings_train.npy'))
embeddings_test = list(np.load('passage_embeddings_test.npy'))

df_train['passage_embeddings'] = embeddings_train
df_test['passage_embeddings'] = embeddings_test

In [93]:
df_train.head()

,query_text,passage_text,query_emb,label,passage_embeddings
0,Когда был спущен на воду первый миноносец «Спо...,Зачислен в списки ВМФ СССР 19 августа 1952 год...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",1,"[0.41186523, -0.25048828, 0.09710693, 0.462646..."
1,Когда был спущен на воду первый миноносец «Спо...,Стерегу́щий — русский миноносец типа «Сокол». ...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0,"[-0.15588379, 0.1348877, -0.4885254, 0.4733886..."
2,Когда был спущен на воду первый миноносец «Спо...,Эскадренный миноносец заложен в 1900 году на Н...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0,"[0.20922852, -0.09442139, -0.4675293, 0.406005..."
3,Когда был спущен на воду первый миноносец «Спо...,10 октября 1937 года эсминец был спущен на вод...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0,"[0.60839844, -0.43481445, -0.5151367, 0.266845..."
4,Когда был спущен на воду первый миноносец «Спо...,В 1901 году миноносец «Бодрый» был зачислен в ...,"[0.46318358, 0.053662203, -0.4220398, 0.277551...",0,"[0.17834473, -0.2680664, -0.55371094, 0.139404..."


In [94]:
df_test.head()

,query_text,passage_text,query_emb,label,passage_embeddings
0,Когда начался Кари́бский кризис?,"Кари́бский кризис — исторический термин, опред...","[0.044620123, -0.035441313, -0.04824933, 0.212...",1,"[-0.025146484, -0.074401855, 0.14025879, 0.001..."
1,Когда начался Кари́бский кризис?,"Политическое, дипломатическое и военное против...","[0.044620123, -0.035441313, -0.04824933, 0.212...",1,"[0.23913574, 0.29492188, 0.2644043, 0.01377105..."
2,Когда начался Кари́бский кризис?,В октябре 1962 года произошел Карибский кризис...,"[0.044620123, -0.035441313, -0.04824933, 0.212...",1,"[0.19262695, 0.40600586, -0.014289856, 0.07025..."
3,Когда начался Кари́бский кризис?,"Первоначально, после победы революции на Кубе ...","[0.044620123, -0.035441313, -0.04824933, 0.212...",0,"[-0.40478516, -0.00012350082, -0.01474762, -0...."
4,Когда начался Кари́бский кризис?,"Невозможно однозначно утверждать, стало ли уда...","[0.044620123, -0.035441313, -0.04824933, 0.212...",0,"[-0.004085541, -0.036743164, 0.1541748, -0.310..."


In [95]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33921 entries, 0 to 33920
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   query_text          33921 non-null  object
 1   passage_text        33921 non-null  object
 2   query_emb           33921 non-null  object
 3   label               33921 non-null  int64 
 4   passage_embeddings  33921 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


In [96]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13100 entries, 0 to 13099
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   query_text          13100 non-null  object
 1   passage_text        13100 non-null  object
 2   query_emb           13100 non-null  object
 3   label               13100 non-null  int64 
 4   passage_embeddings  13100 non-null  object
dtypes: int64(1), object(4)
memory usage: 511.8+ KB


# Обучение модели

## Обучение модели линейной регрессии на трейне

In [98]:
# Объединяем query_emb и passage_embeddings в один массив фичей
X_train = np.hstack((np.stack(df_train['query_emb'].values), np.stack(df_train['passage_embeddings'].values)))
y_train = df_train['label'].values

X_test = np.hstack((np.stack(df_test['query_emb'].values), np.stack(df_test['passage_embeddings'].values)))
y_test = df_test['label'].values

# Создаем и обучаем логистическую регрессию
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [99]:
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка качества модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7232
